In [1]:
!ls

sample_data  test_iemocap.csv  train_iemocap.csv  val_iemocap.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

# Set the path
zip_path = '/content/drive/MyDrive/iemocap_audio.zip'
extract_to = '/content/iemocap/'

# Create target folder
os.makedirs(extract_to, exist_ok=True)

# Extract the zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"✅ Unzipped to {extract_to}")


✅ Unzipped to /content/iemocap/


In [4]:
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install torch==2.0.0 torchvision==0.15.1
!pip install transformers==4.31.0
!pip install torchaudio==2.0.1
!pip install tqdm==4.66.2
!pip install nlpaug==1.1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import pandas as pd
import librosa
from IPython.display import Audio

# Load CSV
csv_path = '/content/test_iemocap.csv'
df = pd.read_csv(csv_path)

# Get the first audio file path
audio_path = df['filepath'].iloc[0]

# Load audio
waveform, sr = librosa.load(audio_path, sr=None)

# Play audio
Audio(waveform, rate=sr)


In [2]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
!pip install peft==0.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import HubertModel, Wav2Vec2FeatureExtractor, BertModel, BertTokenizer
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import nlpaug.augmenter.word as naw
import random
from peft import LoraConfig, get_peft_model, PeftModel
import json
from scipy.stats import pearsonr

# Dataset (unchanged)
class MultimodalIEMOCAPDataset(Dataset):
    def __init__(self, csv_file, audio_feature_extractor, text_tokenizer, augment=True, max_audio_samples=128000):
        self.df = pd.read_csv(csv_file)
        self.audio_feature_extractor = audio_feature_extractor
        self.text_tokenizer = text_tokenizer
        self.augment = augment
        self.text_augmenter = naw.SynonymAug(aug_p=0.3) if augment else None
        self.max_audio_samples = max_audio_samples

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['filepath']
        transcript = self.df.iloc[idx]['transcription']
        arousal_label = self.df.iloc[idx]['EmoAct'].astype(np.float32)
        dominance_label = self.df.iloc[idx]['EmoDom'].astype(np.float32)

        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file missing: {audio_path}")

        try:
            audio, sr = torchaudio.load(audio_path)
            if audio.abs().mean() < 1e-5:
                raise ValueError(f"Silent audio: {audio_path}")
        except Exception as e:
            raise RuntimeError(f"Error loading audio {audio_path}: {str(e)}")

        if sr != 16000:
            audio = torchaudio.transforms.Resample(sr, 16000)(audio)

        audio = audio.squeeze(0)  # Remove channel dim if mono
        if audio.dim() > 1:
            audio = audio[0]  # Take first channel if stereo

        if audio.size(0) > self.max_audio_samples:
            audio = audio[:self.max_audio_samples]
        elif audio.size(0) < self.max_audio_samples:
            audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))

        if self.augment and random.random() < 0.5:
            noise = torch.randn_like(audio) * 0.005
            audio = audio + noise
            try:
                speed_factor = random.uniform(0.9, 1.1)
                effect = torch.tensor(audio).unsqueeze(0)
                augmented_audio, new_sr = torchaudio.sox_effects.apply_effects_tensor(
                    effect,
                    sample_rate=16000,
                    effects=[["speed", str(speed_factor)], ["rate", "16000"]]
                )
                audio = augmented_audio.squeeze(0)
                if audio.size(0) > self.max_audio_samples:
                    audio = audio[:self.max_audio_samples]
                elif audio.size(0) < self.max_audio_samples:
                    audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))
            except Exception as e:
                print(f"Speed augmentation failed for {audio_path}: {e}")

        audio = audio.numpy()

        if self.augment and self.text_augmenter and random.random() < 0.3:
            try:
                transcript = self.text_augmenter.augment(transcript)[0]
            except Exception as e:
                print(f"Text augmentation failed for {audio_path}: {e}")

        audio_inputs = self.audio_feature_extractor(
            audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=False,
            truncation=False
        )

        text_inputs = self.text_tokenizer(
            transcript,
            padding=False,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        return {
            'audio_values': audio_inputs['input_values'].squeeze(0),
            'input_ids': text_inputs['input_ids'].squeeze(0),
            'attention_mask': text_inputs['attention_mask'].squeeze(0)
        }, torch.tensor([arousal_label, dominance_label])

# Dynamic collation (unchanged)
def dynamic_collate_fn(batch):
    inputs, labels = zip(*batch)
    audio_values = [item['audio_values'] for item in inputs]
    input_ids = [item['input_ids'] for item in inputs]
    attention_masks = [item['attention_mask'] for item in inputs]

    audio_values_padded = pad_sequence(audio_values, batch_first=True, padding_value=0.0)
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    labels_stacked = torch.stack(labels)

    return {
        'audio_values': audio_values_padded,
        'input_ids': input_ids_padded,
        'attention_mask': attention_masks_padded
    }, labels_stacked

# AD Model (unchanged)
class MultimodalArousalDominanceModel(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()
        self.hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960")
        hubert_lora_config = LoraConfig(
            r=4,
            lora_alpha=8,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=dropout,
            bias="none"
        )
        self.hubert = get_peft_model(self.hubert, hubert_lora_config)
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        bert_lora_config = LoraConfig(
            r=4,
            lora_alpha=8,
            target_modules=["query", "value"],
            lora_dropout=dropout,
            bias="none"
        )
        self.text_encoder = get_peft_model(self.text_encoder, bert_lora_config)
        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=audio_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim * 4,
                dropout=dropout,
                batch_first=True
            ) for _ in range(num_layers)
        ])
        self.audio_layer_norm = nn.LayerNorm(audio_dim)
        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, 1)
        )
        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)
        self.audio_to_text_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads // 2,
            dropout=dropout,
            batch_first=True
        )
        self.text_to_audio_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads // 2,
            dropout=dropout,
            batch_first=True
        )
        self.audio_gate = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Sigmoid()
        )
        self.text_gate = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Sigmoid()
        )
        self.fusion_layer_arousal = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim)
        )
        self.fusion_layer_dominance = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim)
        )
        self.shared_fc_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.shared_fc_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.output_branch_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim // 2, 1)
        )
        self.output_branch_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim // 2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_values, input_ids, attention_mask):
        audio_features = self.hubert(audio_values).last_hidden_state
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)
        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)
        audio_repr = self.audio_layer_norm(audio_repr)
        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_repr = text_outputs.last_hidden_state
        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)
        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj,
            key=text_proj,
            value=text_proj,
            key_padding_mask=(1 - attention_mask).bool()
        )
        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj,
            key=audio_proj,
            value=audio_proj,
            key_padding_mask=(~audio_mask).bool()
        )
        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)
        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)
        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value
        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count
        fused = torch.cat([pooled_audio, pooled_text], dim=1)
        joint_repr_arousal = self.fusion_layer_arousal(fused)
        joint_repr_dominance = self.fusion_layer_dominance(fused)
        shared_arousal = self.shared_fc_arousal(joint_repr_arousal)
        shared_dominance = self.shared_fc_dominance(joint_repr_dominance)
        output_arousal = self.output_branch_arousal(shared_arousal)
        output_dominance = self.output_branch_dominance(shared_dominance)
        scaled_arousal = 1.0 + 4.0 * torch.sigmoid(output_arousal)
        scaled_dominance = 1.0 + 4.0 * torch.sigmoid(output_dominance)
        return scaled_arousal, scaled_dominance

# Save LoRA weights (unchanged)
def save_lora_weights(model, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    hubert_lora_dir = os.path.join(save_dir, "hubert_lora")
    model.hubert.save_pretrained(hubert_lora_dir)
    print(f"Saved HuBERT LoRA weights to {hubert_lora_dir}")
    bert_lora_dir = os.path.join(save_dir, "bert_lora")
    model.text_encoder.save_pretrained(bert_lora_dir)
    print(f"Saved BERT LoRA weights to {bert_lora_dir}")
    custom_layers_state = {}
    for name, param in model.named_parameters():
        if not (name.startswith('hubert.') or name.startswith('text_encoder.')):
            custom_layers_state[name] = param
    custom_layers_path = os.path.join(save_dir, "custom_layers.pth")
    torch.save(custom_layers_state, custom_layers_path)
    print(f"Saved custom layers to {custom_layers_path}")
    config = {
        'audio_dim': 768,
        'text_dim': 768,
        'hidden_dim': 192,
        'num_heads': 6,
        'num_layers': 2,
        'dropout': 0.5,
        'hubert_lora_config': {
            'r': 4,
            'lora_alpha': 8,
            'target_modules': ["q_proj", "v_proj"],
            'lora_dropout': 0.5,
            'bias': "none"
        },
        'bert_lora_config': {
            'r': 4,
            'lora_alpha': 8,
            'target_modules': ["query", "value"],
            'lora_dropout': 0.5,
            'bias': "none"
        }
    }
    config_path = os.path.join(save_dir, "config.json")
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"Saved model configuration to {config_path}")
    total_size = 0
    for root, dirs, files in os.walk(save_dir):
        for file in files:
            total_size += os.path.getsize(os.path.join(root, file))
    print(f"Total LoRA checkpoint size: {total_size / (1024*1024):.2f} MB")
    return total_size

# Load LoRA model (unchanged)
def load_lora_model(save_dir, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config_path = os.path.join(save_dir, "config.json")
    with open(config_path, 'r') as f:
        config = json.load(f)
    model = MultimodalArousalDominanceModel(
        audio_dim=config['audio_dim'],
        text_dim=config['text_dim'],
        hidden_dim=config['hidden_dim'],
        num_heads=config['num_heads'],
        num_layers=config['num_layers'],
        dropout=config['dropout']
    )
    hubert_lora_dir = os.path.join(save_dir, "hubert_lora")
    model.hubert = PeftModel.from_pretrained(
        model.hubert.get_base_model(),
        hubert_lora_dir
    )
    bert_lora_dir = os.path.join(save_dir, "bert_lora")
    model.text_encoder = PeftModel.from_pretrained(
        model.text_encoder.get_base_model(),
        bert_lora_dir
    )
    custom_layers_path = os.path.join(save_dir, "custom_layers.pth")
    custom_layers_state = torch.load(custom_layers_path, map_location=device)
    model_state = model.state_dict()
    model_state.update(custom_layers_state)
    model.load_state_dict(model_state)
    model = model.to(device)
    print(f"Loaded LoRA model from {save_dir}")
    return model

# Evaluation function (adapted from second code)
def evaluate_ad_model(model, test_loader, device):
    model.eval()
    criterion = nn.HuberLoss()
    test_loss = 0.0
    test_arousal_loss = 0.0
    test_dominance_loss = 0.0
    num_batches = 0
    all_arousal_preds = []
    all_arousal_labels = []
    all_dominance_preds = []
    all_dominance_labels = []

    with torch.no_grad():
        for batch_inputs, labels in test_loader:
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                arousal_labels = labels[:, 0].to(device)
                dominance_labels = labels[:, 1].to(device)

                arousal_outputs, dominance_outputs = model(audio_values, input_ids, attention_mask)
                arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)
                total_loss = 0.5 * arousal_loss + 0.5 * dominance_loss

                test_loss += total_loss.item()
                test_arousal_loss += arousal_loss.item()
                test_dominance_loss += dominance_loss.item()
                num_batches += 1

                all_arousal_preds.append(arousal_outputs.squeeze().cpu().numpy())
                all_arousal_labels.append(arousal_labels.cpu().numpy())
                all_dominance_preds.append(dominance_outputs.squeeze().cpu().numpy())
                all_dominance_labels.append(dominance_labels.cpu().numpy())

            except Exception as e:
                print(f"Error in test batch: {e}")
                continue

    if num_batches > 0:
        test_loss /= num_batches
        test_arousal_loss /= num_batches
        test_dominance_loss /= num_batches
        all_arousal_preds = np.concatenate(all_arousal_preds)
        all_arousal_labels = np.concatenate(all_arousal_labels)
        all_dominance_preds = np.concatenate(all_dominance_preds)
        all_dominance_labels = np.concatenate(all_dominance_labels)
        arousal_pcc, _ = pearsonr(all_arousal_preds, all_arousal_labels)
        dominance_pcc, _ = pearsonr(all_dominance_preds, all_dominance_labels)
    else:
        test_loss = float('inf')
        test_arousal_loss = float('inf')
        test_dominance_loss = float('inf')
        arousal_pcc = 0.0
        dominance_pcc = 0.0

    return test_loss, test_arousal_loss, test_dominance_loss, arousal_pcc, dominance_pcc

# Training function (unchanged from previous response)
def train_ad_model(model, train_loader, val_loader, num_epochs=10, lr=5e-5, max_norm=0.5, device=torch.device("cpu")):
    model = model.to(device)
    optimizer = optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=1e-2
    )
    criterion = nn.HuberLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=3, T_mult=2, eta_min=1e-6
    )
    best_val_loss = float('inf')
    patience_counter = 0
    max_patience = 7
    arousal_weight = 1.0
    dominance_weight = 1.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_arousal_loss = 0.0
        train_dominance_loss = 0.0
        num_train_batches = 0
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train Arousal-Dominance]")
        for batch_inputs, labels in train_pbar:
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                arousal_labels = labels[:, 0].to(device)
                dominance_labels = labels[:, 1].to(device)
                optimizer.zero_grad()
                arousal_outputs, dominance_outputs = model(audio_values, input_ids, attention_mask)
                arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)
                total_loss = arousal_weight * arousal_loss + dominance_weight * dominance_loss
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                optimizer.step()
                train_loss += total_loss.item()
                train_arousal_loss += arousal_loss.item()
                train_dominance_loss += dominance_loss.item()
                num_train_batches += 1
                train_pbar.set_postfix({
                    'loss': f"{total_loss.item():.4f}",
                    'a_loss': f"{arousal_loss.item():.4f}",
                    'd_loss': f"{dominance_loss.item():.4f}"
                })
            except Exception as e:
                print(f"Error in training batch: {e}")
                continue
        scheduler.step()
        if num_train_batches > 0:
            train_loss /= num_train_batches
            train_arousal_loss /= num_train_batches
            train_dominance_loss /= num_train_batches
            total_task_loss = train_arousal_loss + train_dominance_loss
            if total_task_loss > 0:
                arousal_weight = 0.5 * (train_arousal_loss / total_task_loss)
                dominance_weight = 0.5 * (train_dominance_loss / total_task_loss)
                if train_dominance_loss > train_arousal_loss * 1.2:
                    dominance_weight *= 1.5
                    arousal_weight *= 0.5
                total_weight = arousal_weight + dominance_weight
                arousal_weight /= total_weight
                dominance_weight /= total_weight
                arousal_weight = max(0.25, min(0.75, arousal_weight))
                dominance_weight = max(0.25, min(0.75, dominance_weight))
        model.eval()
        val_loss = 0.0
        val_arousal_loss = 0.0
        val_dominance_loss = 0.0
        num_val_batches = 0
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val Arousal-Dominance]")
        with torch.no_grad():
            for batch_inputs, labels in val_pbar:
                try:
                    audio_values = batch_inputs['audio_values'].to(device)
                    input_ids = batch_inputs['input_ids'].to(device)
                    attention_mask = batch_inputs['attention_mask'].to(device)
                    arousal_labels = labels[:, 0].to(device)
                    dominance_labels = labels[:, 1].to(device)
                    arousal_outputs, dominance_outputs = model(audio_values, input_ids, attention_mask)
                    arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                    dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)
                    total_loss = 0.5 * arousal_loss + 0.5 * dominance_loss
                    val_loss += total_loss.item()
                    val_arousal_loss += arousal_loss.item()
                    val_dominance_loss += dominance_loss.item()
                    num_val_batches += 1
                    val_pbar.set_postfix({
                        'loss': f"{total_loss.item():.4f}",
                        'a_loss': f"{arousal_loss.item():.4f}",
                        'd_loss': f"{dominance_loss.item():.4f}"
                    })
                except Exception as e:
                    print(f"Error in validation batch: {e}")
                    continue
        if num_val_batches > 0:
            val_loss /= num_val_batches
            val_arousal_loss /= num_val_batches
            val_dominance_loss /= num_val_batches
        print(f"Epoch {epoch+1}/{num_epochs} [Arousal-Dominance] Results:")
        print(f"  Train Loss: {train_loss:.4f} (Arousal: {train_arousal_loss:.4f}, Dominance: {train_dominance_loss:.4f})")
        print(f"  Val Loss: {val_loss:.4f} (Arousal: {val_arousal_loss:.4f}, Dominance: {val_dominance_loss:.4f})")
        print(f"  Loss Weights - Arousal: {arousal_weight:.4f}, Dominance: {dominance_weight:.4f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_lora_weights(model, '/content/best_arousal_dominance_lora')
            print(f"  Saved best LoRA model (val_loss: {val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"Early stopping after {epoch+1} epochs without improvement")
                break
    return best_val_loss

# Main execution (updated with test set)
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    train_csv = "/content/train_iemocap.csv"
    val_csv = "/content/val_iemocap.csv"
    test_csv = "/content/test_iemocap.csv"
    audio_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
    text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    train_dataset = MultimodalIEMOCAPDataset(
        csv_file=train_csv,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=True,
        max_audio_samples=128000
    )
    val_dataset = MultimodalIEMOCAPDataset(
        csv_file=val_csv,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )
    test_dataset = MultimodalIEMOCAPDataset(
        csv_file=test_csv,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=dynamic_collate_fn
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=dynamic_collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=dynamic_collate_fn
    )
    model = MultimodalArousalDominanceModel(
        audio_dim=768,
        text_dim=768,
        hidden_dim=192,
        num_heads=6,
        num_layers=2,
        dropout=0.5
    ).to(device)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable percentage: {100 * trainable_params / total_params:.2f}%")
    print("Training Arousal-Dominance model...")
    best_val_loss = train_ad_model(
        model,
        train_loader,
        val_loader,
        num_epochs=10,
        lr=5e-5,
        max_norm=0.5,
        device=device
    )
    print(f"Best validation loss: {best_val_loss:.4f}")
    print("\nTesting model loading...")
    loaded_model = load_lora_model('/content/best_arousal_dominance_lora', device)
    print("Model loaded successfully!")
    print("\nEvaluating on test set...")
    test_loss, test_arousal_loss, test_dominance_loss, arousal_pcc, dominance_pcc = evaluate_ad_model(
        loaded_model, test_loader, device
    )
    print("\nTest Results:")
    print(f"  Overall Test Loss: {test_loss:.4f}")
    print(f"  Arousal Loss: {test_arousal_loss:.4f}, PCC: {arousal_pcc:.4f}")
    print(f"  Dominance Loss: {test_dominance_loss:.4f}, PCC: {dominance_pcc:.4f}")

if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Epoch 8/10 [Arousal-Dominance] Results:
  Train Loss: 0.1498 (Arousal: 0.1117, Dominance: 0.1624)
  Val Loss: 0.1411 (Arousal: 0.1181, Dominance: 0.1640)
  Loss Weights - Arousal: 0.2500, Dominance: 0.7500
Saved HuBERT LoRA weights to /content/best_arousal_dominance_lora/hubert_lora
Saved BERT LoRA weights to /content/best_arousal_dominance_lora/bert_lora
Saved custom layers to /content/best_arousal_dominance_lora/custom_layers.pth
Saved model configuration to /content/best_arousal_dominance_lora/config.json
Total LoRA checkpoint size: 33.48 MB
  Saved best LoRA model (val_loss: 0.1411)


Epoch 9/10 [Val Arousal-Dominance]: 100%|██████████| 126/126 [00:45<00:00,  2.74it/s, loss=0.0960, a_loss=0.0484, d_loss=0.1435]


Epoch 9/10 [Arousal-Dominance] Results:
  Train Loss: 0.1474 (Arousal: 0.1087, Dominance: 0.1602)
  Val Loss: 0.1387 (Arousal: 0.1132, Dominance: 0.1641)
  Loss Weights - Arousal: 0.2500, Dominance: 0.7500
Saved HuBERT LoRA weights to /content/best_arousal_dominance_lora/hubert_lora
Saved BERT LoRA weights to /content/best_arousal_dominance_lora/bert_lora
Saved custom layers to /content/best_arousal_dominance_lora/custom_layers.pth
Saved model configuration to /content/best_arousal_dominance_lora/config.json
Total LoRA checkpoint size: 33.48 MB
  Saved best LoRA model (val_loss: 0.1387)


Epoch 10/10 [Val Arousal-Dominance]: 100%|██████████| 126/126 [00:45<00:00,  2.74it/s, loss=0.1052, a_loss=0.1208, d_loss=0.0895]


Epoch 10/10 [Arousal-Dominance] Results:
  Train Loss: 0.1541 (Arousal: 0.1120, Dominance: 0.1682)
  Val Loss: 0.1439 (Arousal: 0.1138, Dominance: 0.1740)
  Loss Weights - Arousal: 0.2500, Dominance: 0.7500
Best validation loss: 0.1387

Testing model loading...
Loaded LoRA model from /content/best_arousal_dominance_lora
Model loaded successfully!

Evaluating on test set...

Test Results:
  Overall Test Loss: 0.1284
  Arousal Loss: 0.0955, PCC: 0.7780
  Dominance Loss: 0.1614, PCC: 0.6765
